In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ace-dataset/ACE_dataset.csv
/kaggle/input/ace-dataset/ACE_dataset.fasta
/kaggle/input/ace-dataset/features/ACE_ASDC.csv
/kaggle/input/ace-dataset/features/opf_7bit_type_1_features.csv
/kaggle/input/ace-dataset/features/opf_7bit_type_2_features.csv
/kaggle/input/ace-dataset/features/esmv1_feat_ACE.csv
/kaggle/input/ace-dataset/features/ACE_embeddings_prot_t5_xl_bfd.csv
/kaggle/input/ace-dataset/features/esm2_t6_8M_feat_ACE.csv
/kaggle/input/ace-dataset/features/opf_7bit_type_3_features.csv
/kaggle/input/ace-dataset/features/opf_10bit_features.csv
/kaggle/input/ace-dataset/features/ACE_AAC.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-500.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-300.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-50.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-400.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-450.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-350.csv
/kaggle/input/ace-dataset/auto_enco_feat/AEDN-250.csv
/kaggle/in

In [2]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
rnnamp_model = pd.read_csv('/kaggle/input/ace-dataset/ACE_dataset.csv')

In [6]:
X=rnnamp_model['text']
y=np.array(rnnamp_model['label'])


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from keras.saving import register_keras_serializable
from tensorflow.keras import regularizers


In [9]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
'''
# Example data
texts = X_train

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(texts)
test_seq=tokenizer.texts_to_sequences(X_test)
# Pad sequences to a fixed length
maxlen = 100  # Adjust as needed based on your data
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
test=pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
# Convert to NumPy array
X_train = tf.constant(padded_sequences)
X_test = tf.constant(test)
'''

In [10]:
X_train=np.array(X_train)
print(X_train.shape)
X_test=np.array(X_test)

(816,)


In [31]:
def tokenizer(text, max_len):
    dic = {'A': 1, 'G': 2, 'V': 3, 'I': 4, 'L': 5, 'F': 6, 'P': 7, 'Y': 8,
           'M': 9, 'T': 10, 'S': 11, 'H': 12, 'N': 13, 'Q': 14, 'W': 15,
           'K': 16, 'R': 17, 'D': 18, 'E': 19, 'C': 20}
    
    onehot = []
    t = []
    
    for i in range(len(text)):
        row = []
        l = []
        chars = text[i].split()  # split by default splits by any whitespace
        for j in range(max_len):
            if j < len(chars):
                if chars[j] in dic:
                    row.append(dic[chars[j]])
                    r = np.zeros(20)
                    r[dic[chars[j]] - 1] = 1
                else:
                    # Handle unknown characters
                    row.append(0)  # or another default value
                    r = np.ones(20) * -1
            else:
                r = np.ones(20) * -1
                row.append(0)
            l.append(r)
        l = np.array(l)
        onehot.append(l)
        t.append(row)
    
    onehot = np.array(onehot)
    t = np.array(t)
    
    return t, onehot
max_len=30

X_train,onehot_train=tokenizer(X_train,max_len)
X_test,onehot_test=tokenizer(X_test,max_len)

In [18]:
print("X_train:\n", X_train)
print("onehot_train:\n", onehot_train)
print("X_test:\n", X_test)
print("onehot_test:\n", onehot_test)

X_train:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
onehot_train:
 [[[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 ...

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ..

In [19]:
print("X_train:\n", X_train.shape)
print("onehot_train:\n", onehot_train.shape)
print("X_test:\n", X_test.shape)
print("onehot_test:\n", onehot_test.shape)

X_train:
 (816, 30)
onehot_train:
 (816, 30, 20)
X_test:
 (204, 30)
onehot_test:
 (204, 30, 20)


In [14]:
path = "/kaggle/input/ace-dataset/auto_enco_feat/"
data_ = pd.read_csv(path + 'AEDN-100.csv')

data_np = np.array(data_)
data = data_np[:, 1:]
# data = data_.iloc[:, :-1].values
# labels = data_.iloc[:, -1].values

label1=np.ones((394,1))#Value can be changed
label2=np.zeros((626,1))
labels=np.append(label1,label2)

X_train, X_test, y_train, y_test= train_test_split(data,labels, test_size=0.2, random_state=0)
print(X_train.shape)

(816, 100)


In [15]:
def positional_encoding(positions, d):

    # initialize a matrix angle_rads of all the angles
    pos=np.arange(positions)[:, np.newaxis] #Column vector containing the position span [0,1,..., positions]
    k= np.arange(d)[np.newaxis, :]  #Row vector containing the dimension span [[0, 1, ..., d-1]]
    i = k//2
    angle_rads = pos/(10000**(2*i/d)) #Matrix of angles indexed by (pos,i)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    #adds batch axis
    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
@register_keras_serializable()
class TransformerModel(keras.Model):
    def __init__(self, input_vocab_size, d_model, num_heads, ff_dim, rate=0.1, maxlen=50):
        super(TransformerModel, self).__init__()

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.PE = positional_encoding(maxlen, d_model)
        self.transformer_block = TransformerBlock_Encode(d_model, num_heads, ff_dim, rate)
        self.transformer_block2 = TransformerBlock_decode(d_model, num_heads, ff_dim, rate)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation="relu")
        self.fc3 = layers.Dense(256, activation="relu")
        self.fc2 = layers.Dense(1, activation="sigmoid")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        #x = x+self.PE
        y = self.transformer_block(x)
        x = self.transformer_block2(x,y,y)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc3(x)
        return self.fc2(x)

class TransformerBlock_decode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_decode, self).__init__()

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu", kernel_regularizer=regularizers.l2(0.01)), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, inputs, q, k, training=None):
        attn_output = self.att(inputs, inputs, inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))

        attn_output1 = self.att1(q, k, out1)
        out2 = self.layernorm2(out1 + self.dropout2(attn_output1, training=training))

        ffn_output = self.ffn(out2)
        return self.layernorm3(out2 + self.dropout3(ffn_output, training=training))


class TransformerBlock_Encode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_Encode, self).__init__()
        self.con= layers.Conv1D(256,5,padding='same')
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu", kernel_regularizer=regularizers.l2(0.01)), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=None):
        inputs = self.con(inputs)
        attn_output = self.att(inputs, inputs, inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))
    
# class TransformerBlock_decode(layers.Layer):
#     def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock_decode, self).__init__()

#         self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.ffn = keras.Sequential(
#             [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
#         )
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(rate)
#         self.dropout2 = layers.Dropout(rate)
#         self.dropout3 = layers.Dropout(rate)

#     def call(self, inputs, q, k, training=None):
#         attn_output = self.att(inputs, inputs, inputs)
#         out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))

#         attn_output1 = self.att1(q, k, out1)
#         out2 = self.layernorm2(out1 + self.dropout2(attn_output1, training=training))

#         ffn_output = self.ffn(out2)
#         return self.layernorm3(out2 + self.dropout3(ffn_output, training=training))


# class TransformerBlock_Encode(layers.Layer):
#     def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock_Encode, self).__init__()
#         self.con= layers.Conv1D(256,5,padding='same')
#         self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
#         self.ffn = keras.Sequential(
#             [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
#         )
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(rate)
#         self.dropout2 = layers.Dropout(rate)

#     def call(self, inputs, training=None):
#         inputs = self.con(inputs)
#         attn_output = self.att(inputs, inputs, inputs)
#         out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        
#         ffn_output = self.ffn(out1)
#         return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))


# Define your data loading and preprocessing here
# Example: X_train, y_train = load_data_and_preprocess()

# Define the model
input_vocab_size = 1024  # Replace with the actual vocabulary size
d_model = 256
num_heads = 8
ff_dim = 256

model = TransformerModel(input_vocab_size, d_model, num_heads, ff_dim)
initial_learning_rate = 0.0001

optimizer = Adam(learning_rate=initial_learning_rate, clipvalue=0.5)
# Compile the model
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"])

filepath = "/kaggle/working/"
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
            tf.keras.callbacks.ModelCheckpoint(filepath='.weights.h5', monitor='val_accuracy', save_best_only=True, mode='auto', save_weights_only=True),
            lr_scheduler]
# Train the model
history = model.fit(X_train,y_train,epochs = 100,batch_size=32,validation_split=0.1,callbacks=[callback],verbose=1,shuffle= True)

Epoch 1/100
 1/23 ━━━━━━━━━━━━━━━━━━━━ 5:18 14s/step - accuracy: 0.5625 - loss: 5.9447

In [ ]:
model.load_weights('/kaggle/input/amap/tensorflow2/model/1/ACE.h5')

In [ ]:
model1=model

In [ ]:
from sklearn import model_selection, metrics
y_pred=model.predict(X_train)
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred, pos_label=1)
auc=metrics.auc(fpr, tpr)

y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('combined train datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_train, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_train, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_train, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_train, cv_preds)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_train, cv_preds)))
print("%s: AUC Score %0.2f%%" % (name, 100*auc))
target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_train, cv_preds, target_names=target_names))

# Predictions Validation Set
print('combined test datasets')
y_pred2=model.predict(X_test)
l=np.zeros(len(y_pred2))
l=l.reshape(-1,1)
l[y_pred2>=0.5]=1
l[y_pred2<0.5]=0
cv_preds2 = l
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_test, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test, cv_preds2)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_test, cv_preds2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred2, pos_label=1)
auc=metrics.auc(fpr, tpr)
print("%s: AUC Score %0.2f%%" % (name, 100*auc))

target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_test, cv_preds2, target_names=target_names))